In [1]:
import tensorflow as tf

import tensorflow.examples.tutorials.mnist.input_data as input_data

mnist = input_data.read_data_sets("MNIST_data/", one_hot=True)


E:\python\python_install\lib\site-packages\h5py\__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.
Instructions for updating:
Please write your own downloading logic.
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting MNIST_data/train-images-idx3-ubyte.gz
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Instructions for updating:
Please use tf.one_hot on tensors.
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz
Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.


In [2]:
def fcn_layer(inputs,
             input_dim,
             output_dim,
             activation=None):
    W = tf.Variable(tf.truncated_normal([input_dim, output_dim], stddev=0.1))
    b = tf.Variable(tf.zeros([output_dim]))
    XWB = tf.matmul(inputs, W) + b
    if activation == None:
        outputs = XWB
    else:
        outputs = activation(XWB)
    return outputs

In [3]:
x = tf.placeholder(tf.float32, [None, 784], name="X")
y = tf.placeholder(tf.float32, [None, 10], name="Y")

In [4]:
H1_NN = 256
H2_NN = 32

In [5]:
h1 = fcn_layer(inputs=x,
              input_dim=784,
              output_dim=H1_NN,
              activation=tf.nn.relu)

h2 = fcn_layer(inputs=h1,
              input_dim=H1_NN,
              output_dim=H2_NN,
              activation=tf.nn.relu)

forward = fcn_layer(inputs=h2,
                   input_dim=H2_NN,
                   output_dim=10)
pred = tf.nn.softmax(forward)

# 训练模型

## 定义损失函数

In [6]:
#交叉熵
loss_function = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(logits=forward,labels=y))

Instructions for updating:

Future major versions of TensorFlow will allow gradients to flow
into the labels input on backprop by default.

See @{tf.nn.softmax_cross_entropy_with_logits_v2}.



## 定义超参数

In [7]:
train_epochs = 30
batch_size = 50
total_batch = int(mnist.train.num_examples/batch_size)
display_step = 2
learning_rate = 0.01

save_step = 5

import os 
ckpt_dir = "./ckpt_dir/"
if not os.path.exists(ckpt_dir):
    os.makedirs(ckpt_dir)
    

## 定义优化器

In [8]:
optimizer = tf.train.AdamOptimizer(learning_rate).minimize(loss_function)

## 定义准确率

In [9]:
correct_prediction = tf.equal(tf.argmax(y, 1), tf.argmax(pred, 1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

## 训练并存储模型

In [15]:
saver = tf.train.Saver()

from time import time
startTime = time()

sess = tf.Session()
sess.run(tf.global_variables_initializer())

for epoch in range(train_epochs):
    for batch in range(total_batch):
        xs, ys = mnist.train.next_batch(batch_size)
        sess.run(optimizer, feed_dict={x: xs, y: ys})
    
    loss, acc = sess.run([loss_function, accuracy], 
                        feed_dict={x: mnist.validation.images,
                                  y: mnist.validation.labels})
    if (epoch+1) % display_step == 0:
        print("Train Epoch:%02d Loss=%.9f Accuracy=%.4f" % (epoch+1, loss, acc))
    
    if (epoch+1) % save_step == 0:
        saver.save(sess, os.path.join(ckpt_dir, 'mnist_h256_h32_model_{:06d}.ckpt'.format(epoch+1)))
        print('mnist_h256_h32_model_{:06d}.ckpt'.format(epoch+1))

saver.save(sess, os.path.join(ckpt_dir,'mnist_h256_h32_model.ckpt'))
print("Model saved!")
           
duration = time() - startTime
print("Train Finished takes:%.2f" % duration)

Train Epoch:02 Loss=0.122702725 Accuracy=0.9690
Train Epoch:04 Loss=0.139723137 Accuracy=0.9666
mnist_h256_h32_model_000005.ckpt
Train Epoch:06 Loss=0.169960245 Accuracy=0.9628
Train Epoch:08 Loss=0.137508050 Accuracy=0.9720
Train Epoch:10 Loss=0.154392868 Accuracy=0.9698
mnist_h256_h32_model_000010.ckpt
Train Epoch:12 Loss=0.167015105 Accuracy=0.9678
Train Epoch:14 Loss=0.158029646 Accuracy=0.9702
mnist_h256_h32_model_000015.ckpt
Train Epoch:16 Loss=0.226884991 Accuracy=0.9680
Train Epoch:18 Loss=0.217507422 Accuracy=0.9656
Train Epoch:20 Loss=0.219660297 Accuracy=0.9718
mnist_h256_h32_model_000020.ckpt
Train Epoch:22 Loss=0.211933613 Accuracy=0.9734
Train Epoch:24 Loss=0.252761781 Accuracy=0.9668
mnist_h256_h32_model_000025.ckpt
Train Epoch:26 Loss=0.262725711 Accuracy=0.9728
Train Epoch:28 Loss=0.466935545 Accuracy=0.9620
Train Epoch:30 Loss=0.307813585 Accuracy=0.9712
mnist_h256_h32_model_000030.ckpt
Model saved!
Train Finished takes:110.19


# 模型评估

In [16]:
accu_test = sess.run(accuracy, feed_dict={x: mnist.test.images,
                                          y: mnist.test.labels})
print("Test Accuracy:", accu_test)

Test Accuracy: 0.9695


# 进行预测